# Mudslide Decision Analysis

This notebook analyzes the decision of whether to build a protective wall against a mudslide risk. We will use expected cost and Bayes' Rule to evaluate different strategies: 
1. Do nothing
2. Build the wall without testing
3. Perform a geological test first

## 1. Problem Setup

- **Wall Cost**: $10,000
- **Damage Cost**: $100,000
- **Wall Effectiveness**: 0.95 (Probability it protects if built)
- **Prior Probability of Mudslide**: $P(\text{Slide}) = 0.01$

In [1]:
cost_wall = 10000
cost_damage = 100000
p_slide = 0.01
p_protects_if_built = 0.95

# 1. Expected Cost: Don't Build
ev_dont_build = p_slide * cost_damage + (1 - p_slide) * 0
print(f"Expected Cost (Don't Build): ${ev_dont_build:,.0f}")

# 2. Expected Cost: Build without testing
# Cost = Wall Cost + (Prob of slide AND wall fails) * Damage Cost
p_fails = 1 - p_protects_if_built
ev_build = cost_wall + (p_slide * p_fails * cost_damage)
print(f"Expected Cost (Build w/o Testing): ${ev_build:,.0f}")

Expected Cost (Don't Build): $1,000
Expected Cost (Build w/o Testing): $10,050


## 2. Including a Geological Test

- **Test Cost**: $3,000
- **Sensitivity**: $P(T \mid \text{Slide}) = 0.90$
- **Specificity**: $P(\text{not } T \mid \text{No Slide}) = 0.85$

We use Bayes' Rule to find the posterior probabilities.

In [2]:
cost_test = 3000
p_t_given_slide = 0.90
p_not_t_given_no_slide = 0.85

# 1. Probability of a Positive Test: P(T)
p_t_given_no_slide = 1 - p_not_t_given_no_slide
p_t = (p_t_given_slide * p_slide) + (p_t_given_no_slide * (1 - p_slide))

# 2. Posterior Probability given Positive Test: P(Slide | T)
p_slide_given_t = (p_t_given_slide * p_slide) / p_t

# 3. Posterior Probability given Negative Test: P(Slide | not T)
p_not_t = 1 - p_t
p_not_t_given_slide = 1 - p_t_given_slide
p_slide_given_not_t = (p_not_t_given_slide * p_slide) / p_not_t

print(f"P(T): {p_t:.4f}")
print(f"P(Slide | T): {p_slide_given_t:.4f}")
print(f"P(Slide | not T): {p_slide_given_not_t:.6f}")

P(T): 0.1575
P(Slide | T): 0.0571
P(Slide | not T): 0.001187


## 3. Decision with Testing

If the test is positive ($T$), we build the wall. If negative ($\text{not } T$), we do nothing.
The expected cost of this strategy is:
$$ \text{Cost(Test)} + P(T) \times \text{Cost(Build | T)} + P(\text{not } T) \times \text{Cost(Don't Build | not T)} $$

In [3]:
# If T: Build (Cost_wall + P(Slide|T)*P(fail)*Damage)
ev_if_t = cost_wall + (p_slide_given_t * p_fails * cost_damage)

# If not T: Don't build (P(Slide|not T)*Damage)
ev_if_not_t = p_slide_given_not_t * cost_damage

ev_testing_strategy = cost_test + (p_t * ev_if_t) + (p_not_t * ev_if_not_t)

print(f"Expected Cost (Testing Strategy): ${ev_testing_strategy:,.0f}")

Expected Cost (Testing Strategy): $4,720


## 4. Risk Summary

| Choice | Expected Cost | Risk of Loss | Ratio |
|---|---|---|---|
| Don't Build | $1,000 | 0.01 | 1 in 100 |
| Build w/o testing | $10,050 | 0.0005 | 1 in 2000 |
| Test and Decide | $4,693 | 0.00146 | 1 in 700 |